# LLMs check

In [ ]:
from huggingface_hub import login

login("PLACE_YOR_TOKEN")

In [2]:
import torch

In [18]:
import nbformat

# Load the notebook
with open("phiNANAgent.ipynb", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Remove widget metadata if present
if 'widgets' in nb.metadata:
    del nb.metadata['widgets']

# Save the cleaned notebook
with open("phiNANAgent.ipynb", "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("Widget metadata removed.")

Widget metadata removed.


## Quantized Mistral-7B-Instruct-v0.2

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
prompt = """[INST] What is the NYSE tikers of these companies?
Apple, Tesla, Nvidia, Microsoft
[/INST]"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
print(decoded_output.split("[/INST]")[-1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 The NYSE tickers for the companies you mentioned are as follows:

* Apple Inc.: AAPL
* Tesla, Inc.: TSLA
* Nvidia Corporation: NVDA
* Microsoft Corporation: MSFT


## Flan-t5-small

In [3]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

<pad> Wie ich er bitten?</s>


In [9]:
input_text = f"summarize the following text: {text}"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
with torch.no_grad():
    outputs = model.generate(input_ids)
    decoded_output = tokenizer.decode(outputs[0])

print(decoded_output)

Token indices sequence length is longer than the specified maximum sequence length for this model (23805 > 512). Running this sequence through the model will result in indexing errors


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.67 GiB. GPU 0 has a total capacity of 6.00 GiB of which 4.45 GiB is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 499.95 MiB is allocated by PyTorch, and 50.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# finbert-tone

In [10]:
from transformers import pipeline

sentiment_analyzer = pipeline(
    "text-classification",
    model="yiyanghkust/finbert-tone",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Device set to use cuda


In [11]:
text = "According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing"

In [12]:
with torch.no_grad():
    result = sentiment_analyzer(text)
label = result[0]['label']
label

'Neutral'

# Datasets

## sentiment-analysis-for-financial-news
https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ankurzing/sentiment-analysis-for-financial-news")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/ankurzing/sentiment-analysis-for-financial-news/versions/5


In [27]:
import pandas as pd
import os

df = pd.read_csv(os.path.join(path,"all-data.csv"), encoding="latin-1", names=["reference", "text"])

print(df.head())

       goal                                               text
0   neutral  According to Gran , the company has no plans t...
1   neutral  Technopolis plans to develop in stages an area...
2  negative  The international electronic industry company ...
3  positive  With the new production plant the company woul...
4  positive  According to the company 's updated strategy f...


### Bert evaluation

In [40]:
def summarize_batch(texts, summarizer, batch_size=8):
    return [out["label"] for out in summarizer(texts, batch_size=batch_size)]

In [42]:
df['summary'] = summarize_batch(df['text'].tolist(), sentiment_analyzer, batch_size=4)
df["summary"] = df["summary"].str.lower()

In [69]:
from sklearn.metrics import classification_report, accuracy_score

y_true = df['goal']
y_pred = df['summary']

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))

Accuracy: 0.7926124638877424

Classification Report:
               precision    recall  f1-score   support

    negative       0.79      0.68      0.73       604
     neutral       0.79      0.92      0.85      2879
    positive       0.81      0.57      0.67      1363

    accuracy                           0.79      4846
   macro avg       0.80      0.73      0.75      4846
weighted avg       0.79      0.79      0.78      4846



## Financial Narrative Summaristion 2022
https://www.kaggle.com/datasets/aryankashyapnaveen/financial-narrative-summaristion-2022

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("aryankashyapnaveen/financial-narrative-summaristion-2022")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/aryankashyapnaveen/financial-narrative-summaristion-2022/versions/1


In [6]:
import pandas as pd
import os

df = pd.read_csv(os.path.join(path,"fns_2020.csv"), encoding="latin-1")#, names=["Gold Summaries", "Annual Reports"])

print(df.head())

                                      Gold Summaries  \
0   chairmanâs statement\nit has always been yo...   
1   pan african resources plc integrated annual r...   
2   note the operating profit operating margin ad...   
3   significant progress in the year under review...   
4   anite annual report and accounts 2012\n08\nou...   

                                      Annual Reports  
0   mining corporation plc annual report  account...  
1   the african focused precious metals producer\...  
2   wwwtribalgroupcouk\ntribal group plc\n8791 ne...  
3   msb annual report 2005\nintegrated recruitmen...  
4   anite plc\nannual report and accounts 2012  6...  


In [8]:
text = df["Annual Reports"].iloc[0]

In [46]:
predictions = df["summary"].tolist()
references = df["goal"].tolist()

In [61]:
from bert_score import score
import evaluate

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

rouge_results = rouge.compute(predictions=predictions, references=references)
bleu_results = bleu.compute(predictions=predictions, references=references)
P, R, F1 = score(predictions, references, lang="en", verbose=True)
metrics = {
    "ROUGE": {k: round(v * 100, 2).item() for k, v in rouge_results.items()},
    "BLEU:": round(bleu_results["bleu"] * 100, 2),
    "BERTScore":{
        "Precision":P.mean(),
        "Recall": R.mean(),
        "F1": F1.mean()
    }
}

print(metrics)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/76 [00:00<?, ?it/s]

done in 1.50 seconds, 3220.67 sentences/sec
{'ROUGE': {'rouge1': 79.25, 'rouge2': 0.0, 'rougeL': 79.28, 'rougeLsum': 79.26}, 'BLEU:': 0.0, 'BERTScore': {'Precision': tensor(0.9885), 'Recall': tensor(0.9885), 'F1': tensor(0.9885)}}


In [63]:
bleu_results

{'bleu': 0.0,
 'precisions': [0.0, 0.0, 0.0, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 4846,
 'reference_length': 4846}

In [52]:


P, R, F1 = score(predictions, references, lang="en", verbose=True)

# Среднее по всем примерам
print(f"BERTScore - Precision: {P.mean():.3f}, Recall: {R.mean():.3f}, F1: {F1.mean():.3f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/76 [00:00<?, ?it/s]

done in 1.45 seconds, 3350.33 sentences/sec
BERTScore - Precision: 0.989, Recall: 0.989, F1: 0.989
